In [6]:
from IPython import display
import os
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random as python_random
import tensorflow as tf
import seaborn as sns
import math
from sklearn.metrics import classification_report, confusion_matrix

np.random.seed(42)
tf.random.set_seed(42)

In [13]:
# ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from PIL import ImageFilter


height, width = 32, 32
batch_size=64
data_directory = "./dataset/Validation/"

# missing O
directories = os.listdir(data_directory)

def generate_data(DIR):
    datagen = ImageDataGenerator(rescale=1./255.)

    generator = datagen.flow_from_directory(
        DIR,
        batch_size=batch_size,
        shuffle=True,
        seed=42,
        target_size=(height, width),
        classes=directories
    )
    return generator

TRAINING_DIR = './dataset/Validation/'
TESTING_DIR = './dataset/Train/'

train_generator = generate_data(TRAINING_DIR)
test_generator = generate_data(TESTING_DIR)

total_image = np.concatenate([train_generator.labels,test_generator.labels])



Found 22524 images belonging to 39 classes.
Found 834036 images belonging to 39 classes.


In [14]:
#define VGG16 Model
from tensorflow.keras.layers.experimental import preprocessing

tf.keras.backend.clear_session()
input_shape = (height, width, 3)
base_model = tf.keras.applications.vgg16.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=input_shape
)
base_model.trainable = False #change to true and comment the difference

model_vgg16 = tf.keras.Sequential()
model_vgg16.add(base_model)
model_vgg16.add(tf.keras.layers.GlobalAveragePooling2D())

# model_vgg16.add(tf.keras.layers.Flatten())
model_vgg16.add(tf.keras.layers.Dense(256, activation='relu'))
model_vgg16.add(tf.keras.layers.Dropout(0.5))
model_vgg16.add(tf.keras.layers.Dense(256, activation='relu'))
model_vgg16.add(tf.keras.layers.Dropout(0.5))

model_vgg16.add(tf.keras.layers.Dense(3, activation='softmax'))

model_vgg16.compile(loss='SparseCategoricalCrossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model_vgg16.summary()
model_vgg16.save("model.h5")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                        

In [15]:
# checkpoint --> mode max and save best only
# earlystop --.> mode min and i want to restore the best weights and also patients 5 or less
from keras.callbacks import EarlyStopping
checkpoint = tf.keras.callbacks.ModelCheckpoint(
             filepath='./model.h5',
             monitor='val_accuracy',
             mode='min',
            save_best_only=True,
            verbose=1)
early =EarlyStopping(monitor='val_loss', mode='min',restore_best_weights=True, verbose=1, patience=1)

callbacks_list = [checkpoint,early]

history = model_vgg16.fit(
        train_generator,
        validation_data = test_generator,
        #steps_per_epoch=10,
        epochs=20,
        shuffle=False,
        verbose=True,
        callbacks=callbacks_list
        )


Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "c:\Users\ma407\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Users\ma407\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "c:\Users\ma407\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\ma407\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance

  File "c:\Users\ma407\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 737, in start

  File "c:\Users\ma407\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\ma407\AppData\Local\Programs\Python\P

In [ ]:
# plot learning curve
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#get results
train_result = model_vgg16.evaluate(train_generator)
test_result = model_vgg16.evaluate(test_generator)

no_augmented_df = pd.DataFrame(zip(train_result,test_result),columns=['Train','Val'],index=['Loss','Acc'])
no_augmented_df

In [ ]:
#adding image augmentation
def generate_data_augmented(DIR):
    datagen=ImageDataGenerator(
      rescale=1./255,
      zoom_range=0.1,
      rotation_range=20,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True
    )
         # data generator
        # rescale , zoom with range 0.1 rotate 20, shift in both directions 0.1 , and horizontal flip

    generator = datagen.flow_from_directory(
        TRAINING_DIR,
        batch_size=batch_size,
        seed=42,
        class_mode='binary',
        target_size=(height, width),
        classes={'Normal': 0, 'Viral Pneumonia': 1,'Covid': 2}
    )
    return generator

aug_train_generator = generate_data_augmented(TRAINING_DIR)



In [ ]:
# checkpoint --> mode max and save best only
# earlystop --.> mode min and i want to restore the best weights and also patients 5 or less

checkpoint = tf.keras.callbacks.ModelCheckpoint(
             filepath='/content/model.h5',
             monitor='val_accuracy',
             mode='min',
            save_best_only=True,
            verbose=1)
early =EarlyStopping(monitor='val_loss', mode='min',restore_best_weights=True, verbose=1, patience=1)

callbacks_list = [checkpoint,early]

history = model_vgg16.fit(
        aug_train_generator,
        validation_data = test_generator,
        #steps_per_epoch=10,
        epochs=10,
        shuffle=False,
        verbose=True,
        callbacks=callbacks_list)

In [ ]:
train_result = model_vgg16.evaluate(train_generator)
test_result = model_vgg16.evaluate(test_generator)

augmented_df = pd.DataFrame(zip(train_result,test_result),columns=['Train','Val'],index=['Loss','Acc'])
augmented_df